In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df = df.drop(['RowNumber', "CustomerId" , "Surname" ],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [6]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [7]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## Combine one hot encoder columns with the original data
df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
## DiVide the dataset into indepent and dependent features
X = df.drop('EstimatedSalary',axis=1)
y = df['EstimatedSalary']

In [10]:
## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# **ANN REGRESSION**

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
## Build Our ANN Model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected with input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1) ## Output Layer
]
    
)

In [14]:
# Compile the model
model.compile(optimizer='adam', loss="mean_absolute_error", metrics=['mae'])


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [17]:
## Set up early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [18]:
### Train the model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorflow_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 100366.7578 - mae: 100366.7578 - val_loss: 98475.8594 - val_mae: 98475.8594
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99491.0859 - mae: 99491.0859 - val_loss: 96723.0781 - val_mae: 96723.0781
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 96467.5469 - mae: 96467.5469 - val_loss: 92302.5781 - val_mae: 92302.5781
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 90604.2500 - mae: 90604.2500 - val_loss: 85108.3906 - val_mae: 85108.3906
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 82270.1328 - mae: 82270.1328 - val_loss: 76110.4453 - val_mae: 76110.4453
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 72747.2734 - mae: 72747.2734 - val_loss: 67184.5469 - val_mae: 67184.5469
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 64042.2930 - mae: 64042.293

In [19]:
## Load Tensorboard Extension
%load_ext tensorboard


In [23]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 4464), started 0:00:25 ago. (Use '!kill 4464' to kill it.)

In [21]:
## Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae:.2f}')  

63/63 [==============================] - 0s 789us/step - loss: 50144.2188 - mae: 50144.2188
Test MAE: 50144.22


In [22]:
model.save('regression_model.h5')

c:\Users\vikas\OneDrive\Desktop\Machine Learning\Estimated-Salary-Prediction\viks\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
